# 准备环境

In [ ]:
#@title 挂载谷歌硬盘(可选)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!echo "build trime"
!uname -a
!git --version
!echo "=================apt upgrade====================="
!apt-get update

In [ ]:
#@markdown #1. 安装依赖
#@markdown 安装指定版本的Java：
JAVA_VERSION = "17" # @param ["21", "17", "11"]
!apt install openjdk-{JAVA_VERSION}-jdk
!apt-get install android-sdk
!apt-get install ninja-build
!apt-get install libtool
!apt install -y clang-format

#@markdown #2. 安装并配置Android命令行工具

# 路径变量
ANDROID_SDK_DIR = !whereis android-sdk
ANDROID_SDK_DIR = ANDROID_SDK_DIR[0].split(": ")[1]

#  下载并解压
# @markdown 输入android命令行工具的下载地址（ **如果可以使用就不用修改** ）：

ANDROID_CMD_TOOLS_URL = "https://dl.google.com/android/repository/commandlinetools-linux-11076708_latest.zip"  #@param {type: "string"}
!curl {ANDROID_CMD_TOOLS_URL} -o commandlinetools.zip
!unzip commandlinetools.zip

#  移动到sdk安装目录
!mkdir -p {ANDROID_SDK_DIR}/cmdline-tools/latest
!mv cmdline-tools/* {ANDROID_SDK_DIR}/cmdline-tools/latest

#  接受协议
!yes | {ANDROID_SDK_DIR}/cmdline-tools/latest/bin/sdkmanager --licenses


!echo ">>>>>> 编译环境安装完成"

# 编译Trime

*完整编译一次Trime需要30min以上*

## 1 获取源代码

In [ ]:
%cd /content
#@title Git Clone

import os
import urllib.request

#@markdown 在此处输入源代码仓库地址（一般使用github仓库）：
REPO_URL = "https://github.com/osfans/trime"  #@param {type: "string"}
#@markdown 选择版本/分支（无内容为当前最新）：
GIT_TAG = "v3.2.9" # @param ["", "v3.2.7", "v3.2.8", "v3.2.9", "v3.2.16", "v3.2.17"] {allow-input: true}

# 检查仓库地址是否有效
try:
  urllib.request.urlopen(REPO_URL)
except urllib.error.URLError:
  print(">>>>>> 仓库地址无效！")

# 如果./trime存在，先删除
if os.path.exists("./trime"):
  !rm -rf ./trime

# 开始克隆
!git clone {REPO_URL} trime

%cd trime
if GIT_TAG:
  !git checkout {GIT_TAG}
print("init submodule")
!git submodule update --init --recursive

!echo ">>>>>> 克隆完成"

print("Trime 版本信息")
!git describe --tags --long --always

print("Submodules 版本信息")
submodules = []

with open('.gitmodules', 'r') as file:
    for line in file:
        if line.strip().startswith('path ='):
            submodule_path = line.strip().split('=')[1].strip()
            submodules.append(submodule_path)

print("contains submodules:", submodules)

import subprocess
import os

# 获取当前仓库的根目录
root_dir = subprocess.check_output(['git', 'rev-parse', '--show-toplevel'], text=True).strip()

for submodule in submodules:

    submodule_path = os.path.join(root_dir, submodule)
    print(f"\n{submodule.split('/')[-1]}")

  # 获取提交的 tags 并按提交时间排序
    result = subprocess.run(['git', 'for-each-ref', '--sort=-committerdate', '--format=%(refname:short)', 'refs/tags'], capture_output=True, text=True, cwd=submodule_path)

    # 将输出按行分割并保存为列表
    tags = result.stdout.splitlines()

    # 打印 tags 列表
    print(tags)
    if len(tags)>1:
      filtered_tags = [tag for tag in tags if 'beta' not in tag and 'alpha' not in tag and '.' in tag]
      if len(tags)>len(filtered_tags):
        print("filted tags:\n", filtered_tags)


%cd /content

In [ ]:
#@title 更新submodule为指定版本(可选)
#@markdown **版本号为空的submodule不更新**
%cd /content/trime
#@markdown librime librime-charcode librime-lua librime-octagram
UPDATE_LIBRIME ="1.11.2" # @param ["", '1.11.2', '1.11.1', '1.11.0', '1.10.0'] {allow-input: true}
if UPDATE_LIBRIME:
  !cd app/src/main/jni/librime;git checkout {UPDATE_LIBRIME}


UPDATE_LIBRIME_CHARCODE = True # @param {type:"boolean"}
if UPDATE_LIBRIME_CHARCODE:
  !git submodule update --init --remote  app/src/main/jni/librime-charcode


UPDATE_LIBRIME_LUA = True # @param {type:"boolean"}
if UPDATE_LIBRIME_LUA:
  !git submodule update --init --remote  app/src/main/jni/librime-lua
  !git submodule update --init --remote  app/src/main/jni/librime-lua-deps


UPDATE_LIBRIME_OCTAGRAM = True # @param {type:"boolean"}
if UPDATE_LIBRIME_OCTAGRAM:
  !git submodule update --init --remote  app/src/main/jni/librime-octagram

#@markdown libiconv

UPDATE_LIBICONV = "" # @param ["", 'v1.15', 'v1.13.1', 'v1.13', 'v1.12', 'v1.11.1'] {allow-input: true}
if UPDATE_LIBICONV:
  !cd app/src/main/jni/libiconv;git checkout {UPDATE_LIBICONV}


#@markdown OpenCC
UPDATE_LIBOPENCC = "ver.1.1.7" # @param ["", 'ver.1.1.7', 'ver.1.1.6', 'ver.1.1.5', 'ver.1.1.4', 'ver.1.1.3', 'ver.1.1.1', 'ver.1.1.0', 'ver.1.0.6'] {allow-input: true}
if UPDATE_LIBOPENCC:
  !cd app/src/main/jni/OpenCC;git checkout {UPDATE_LIBOPENCC}

#@markdown boost
UPDATE_LIBBOOST = "" # @param ["", 'boost-1.85.0', 'boost-1.84.0', 'boost-1.83.0', 'boost-1.82.0', 'boost-1.81.0', 'boost-1.80.0', 'boost-1.79.0', 'boost-1.78.0'] {allow-input: true}
if UPDATE_LIBBOOST:
  !cd app/src/main/jni/boost;git checkout {UPDATE_LIBBOOST}



#@markdown snappy
UPDATE_LIBSNAPPY = "" # @param ["", '1.2.0', '1.1.10', '1.1.9', '1.1.8', '1.1.7', '1.1.6', '1.1.5', '1.1.4', '1.1.3'] {allow-input: true}
if UPDATE_LIBSNAPPY:
  !cd app/src/main/jni/snappy;git checkout {UPDATE_LIBSNAPPY}


#@markdown capnproto
UPDATE_LIBCAPN = "" # @param ["", 'v1.0.2', 'v1.0.1.1', 'v1.0.1', 'v1.0.0', 'v0.10.4'] {allow-input: true}
if UPDATE_LIBCAPN:
  !cd app/src/main/jni/capnproto;git checkout {UPDATE_LIBCAPN}



In [ ]:
#@title 更新submodule到上游最新版本(可选)
#@markdown **⚠️更新到不合适的版本可能造成编译失败，因此不建议运行此项**
%cd /content/trime
#@markdown librime librime-charcode librime-lua librime-octagram
UPDATE_LIBRIME = True # @param {type:"boolean"}
if UPDATE_LIBRIME:
  !git submodule update --init --remote  app/src/main/jni/librime
  !git submodule update --init --remote  app/src/main/jni/librime-charcode
  !git submodule update --init --remote  app/src/main/jni/librime-lua
  !git submodule update --init --remote  app/src/main/jni/librime-octagram
# else:
#   !git submodule update --init app/src/main/jni/librime
#   !git submodule update --init app/src/main/jni/librime-charcode
#   !git submodule update --init app/src/main/jni/librime-lua
#   !git submodule update --init app/src/main/jni/librime-octagram

#@markdown libiconv

UPDATE_LIBICONV = False # @param {type:"boolean"}
if UPDATE_LIBICONV:
  !git submodule update --init --remote  app/src/main/jni/libiconv
# else:
#   !git submodule update --init app/src/main/jni/libiconv

#@markdown OpenCC
UPDATE_LIBOPENCC = True # @param {type:"boolean"}
if UPDATE_LIBOPENCC:
  !git submodule update --init --remote  app/src/main/jni/OpenCC
# else:
#   !git submodule update --init app/src/main/jni/OpenCC



#@markdown boost
UPDATE_LIBBOOST = False # @param {type:"boolean"}
if UPDATE_LIBBOOST:
  !git submodule update --init --remote  app/src/main/jni/boost
# else:
#   !git submodule update --init app/src/main/jni/boost


#@markdown snappy
UPDATE_LIBSNAPPY = False # @param {type:"boolean"}
if UPDATE_LIBSNAPPY:
  !git submodule update --init --remote  app/src/main/jni/snappy
# else:
#   !git submodule update --init app/src/main/jni/snappy



#@markdown capnproto
UPDATE_LIBCAPN = False # @param {type:"boolean"}
if UPDATE_LIBCAPN:
  !git submodule update --init --remote  app/src/main/jni/capnproto
# else:
#   !git submodule update --init app/src/main/jni/capnproto




## 2 修改源代码

In [ ]:
#@title 复制谷歌硬盘的jni代码到项目目录（可选）
!cp /content/drive/MyDrive/Trime/librime_jni/*  /content/trime/app/src/main/jni/librime_jni

In [ ]:
#@title 修改Makefile （高版本有效）
# @markdown ### 跳过语法检查
# @markdown 修改文件 /content/trime/Makefile ,去除 clang-format-lint clang-format

MODIFY_MAKEFILE = True # @param {type:"boolean"}
if MODIFY_MAKEFILE:
  !sed -i 's/style-lint//g' /content/trime/Makefile

In [ ]:
#@title 修改build.gradle （低版本有效）

# @markdown 修改文件 /content/trime/app/build.gradle ,减少部分abi类型从而缩短编译时间

BUILD_ARMV7 = False # @param {type:"boolean"}
BUILD_ARMV8 = True # @param {type:"boolean"}
BUILD_X86 = True # @param {type:"boolean"}
BUILD_X64 = False # @param {type:"boolean"}

abi_names = [ ]
if BUILD_ARMV7:
  abi_names.append("armeabi-v7a")
if BUILD_X64:
  abi_names.append("x86_64")
if BUILD_X86:
  abi_names.append("x86")
if BUILD_ARMV8:
  abi_names.append("arm64-v8a")

if len(abi_names)>0:
  build_abi = 'include ' + ", ".join([f'"{item}"' for item in abi_names])
  print("modify to:",build_abi)
  # 修改/content/trime/app/build.gradle中包含`"x86_64"`的一行的内容为变量build_abi的值
  !sed -i 's/.*"x86_64".*/{build_abi}/g' /content/trime/app/build.gradle

In [ ]:
# @title 运行依赖配置脚本（for低版本Trime）
# @markdown **较新的版本（如v3.2.16、v3.2.17）**没有这个脚本，可以**跳过**这一步
%cd /content/trime
# trime的依赖配置脚本（可能是空的）
!bash ./script/dependency.sh
%cd ..

In [ ]:
# @title 编译前配置
# @markdown **⚠️重要： 编译前的一些配置**

# 移动到trime根目录
%cd /content/trime

# @markdown ----
# @markdown (可选) 勾选下方选项以添加git用户名， *旧版TRIME编译中缺失可能会报错*
SET_GIT_USERNAME = True # @param {type:"boolean"}
if SET_GIT_USERNAME:
  GIT_USERNAME = "user_name"  #@param {type: "string"}
  !git config --global user.name {GIT_USERNAME}

# @markdown ----
# 编写local.properties文件，指定SDK路径
!touch local.properties
!echo "sdk.dir={ANDROID_SDK_DIR}" > local.properties
!cat local.properties
!echo ">>>>>> 配置完成"

## 3 编译

In [ ]:
# @title 3.1（可选）清理编译
# @markdown 编译失败 或 编译debug和release版时避免影响

!make clean
!echo ">>>>>> 清理完成，如果仍有残留请手动到左边文件管理中删除编译产物"

### 3.2 （可选）编译 Debug 版

In [ ]:
!make debug
!echo ">>>>>> 编译 (debug) 完成"

### 3.3 编译 Release 版

In [ ]:
# @markdown # Release版需要签名

# 生成release签名密钥
# @markdown **输入签名密钥信息：**
FIRST_AND_LAST_NAME="Tiansuo Li" #@param {type: "string"}
ORGANIZATION_UNIT="Personal" #@param {type: "string"}
ORGANIZATION="Tiansuo Li" #@param {type: "string"}
LOCALITY="Shimokitazawa" #@param {type: "string"}
PROVINCE="Tokyo" #@param {type: "string"}
COUNTRY_CODE="JP" #@param {type: "string"}
SIGNING_KEY_ALIAS="google-colab-build" #@param {type: "string"}
SIGNING_STORE_PASSWORD="114514" #@param {type: "string"}
SIGNING_KEY_PASSWORD="114514" #@param {type: "string"}

# 生成签名密钥
%cd app
if not os.path.exists("my-release-key.keystore"):
  !touch tempfile
  !echo {SIGNING_STORE_PASSWORD} >> tempfile
  !echo {SIGNING_STORE_PASSWORD} >> tempfile
  !echo {FIRST_AND_LAST_NAME} >> tempfile
  !echo {ORGANIZATION_UNIT} >> tempfile
  !echo {ORGANIZATION} >> tempfile
  !echo {LOCALITY} >> tempfile
  !echo {PROVINCE} >> tempfile
  !echo {COUNTRY_CODE} >> tempfile
  !echo "y" >> tempfile
  !cat tempfile | keytool -genkey -v -keystore my-release-key.keystore -alias {SIGNING_KEY_ALIAS} -keyalg RSA -keysize 2048 -validity 10000
  !rm -f tempfile
%cd ..

#@markdown **兼容旧版** （复制一份 `my-release-key.keystore` 到trime文件夹）
兼容旧版 = True #@param {type: "boolean"}
if 兼容旧版:
  !cp app/my-release-key.keystore my-release-key.keystore

# 把密钥信息写入配置文件
if os.path.exists("keystore.properties"):
  !rm -f keystore.properties
!touch keystore.properties
!echo "storeFile=my-release-key.keystore" >> keystore.properties
!echo "keyAlias={SIGNING_KEY_ALIAS}" >> keystore.properties
!echo "storePassword={SIGNING_STORE_PASSWORD}" >> keystore.properties
!echo "keyPassword={SIGNING_KEY_PASSWORD}" >> keystore.properties

# 开始编译
!make release
!echo ">>>>>> 编译 (release) 完成"

## 4 获取结果

运行打包结果的代码（下方的格子）后

*如果不需要打包所有项目，可以去掉勾选，节省时间*


### 4.1 需要获取编译后的完整项目用于开发（完整的trime项目文件夹）

到左侧 `文件` -> `trime.tar` -> `鼠标右键` -> `下载`


### 4.2 需要获取编译后的APK文件（安卓安装包文件）

到左侧 `文件` -> `Trime-APK.tar` -> `鼠标右键` -> `下载`


### 4.3 需要获取编译后的prebuilt文件用于开发（开发时使用的jni部分预编译文件）

到左侧 `文件` -> `prebuilt.tar` -> `鼠标右键` -> `下载`

In [ ]:
# @markdown ##打包结果
%cd ..

# @markdown ----
# @markdown **勾选打包完整项目**
PACK_ALL = False #@param {type: "boolean"}
if PACK_ALL:
  !tar -zcvf trime.tar trime
  !echo ">>>>>> 完整项目打包完成"

# @markdown ----
# @markdown **勾选打包APK安装包**
PACK_APK = True #@param {type: "boolean"}
if PACK_APK:
  !tar -zcvf Trime-APK.tar trime/app/build/outputs/apk
  !echo ">>>>>> apk打包完成"

# @markdown ----
# @markdown **勾选打包prebuilt相关文件**
PACK_PREBUILT = True #@param {type: "boolean"}
if PACK_PREBUILT:
  !tar -zcvf prebuilt.tar trime/app/build/intermediates/stripped_native_libs
  !echo ">>>>>> prebuilt打包完成"

%cd trime

## 5 获取JNI编译结果Ⅱ

In [ ]:
#@title 列出jni编译结果
!find app -name librime_jni.so  -exec ls -lh {} \;

In [ ]:
#@title 复制arm64/x86 jni编译结果到谷歌硬盘
target_path = "/content/drive/MyDrive/Trime/libs/" # @param {type:"string"}
source_path = "app/build/intermediates/stripped_native_libs/release/out/lib/" # @param {type:"string"}
import os
def copy_build_result(source_path, destination_dir, file_name):
  # Check if the file exists
  if os.path.exists(source_path):
    # Create the destination directory if it doesn't exist
    os.makedirs(destination_dir, exist_ok=True)

    # Copy the file
    os.system(f'cp "{source_path}/{file_name}" "{destination_dir}"')
    print(f'Copy from {source_path}/{file_name}')
  else:
    print(f'File not found at {source_path}')

if "abi_names" in globals():
  for abi in abi_names:
    copy_build_result(source_path+abi,target_path+abi, "librime_jni.so")
else:
  for abi in ["x86", "arm64-v8a", "x86_64", "armeabi-v7a" ]:
    copy_build_result(source_path+abi,target_path+abi, "librime_jni.so")
print("finish.")